<a href="https://colab.research.google.com/github/iamsidharthnair/TRAC/blob/master/cnn_gspy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from IPython.display import Image, display

In [2]:
!pip install kaggle


In [3]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"witcher141","key":"9c0d25de1928dc79c9874496d33bbce9"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:


!kaggle datasets download -d tentotheminus9/gravity-spy-gravitational-waves  --force

100% 3.89G/3.89G [02:18<00:00, 28.5MB/s]
100% 3.89G/3.89G [02:18<00:00, 30.1MB/s]


In [6]:
import zipfile
zip_ref = zipfile.ZipFile('gravity-spy-gravitational-waves.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [7]:
import os
train_dir=os.path.join('files/train/train/')
test_dir=os.path.join('files/test/test/')
validation_dir=os.path.join('files/validation/validation/')

In [8]:
import pandas as pd

df = pd.read_csv("files/trainingset_v1d1_metadata.csv") 
df.head()

,event_time,ifo,peak_time,peak_time_ns,start_time,start_time_ns,duration,search,process_id,event_id,peak_frequency,central_freq,bandwidth,channel,amplitude,snr,confidence,chisq,chisq_dof,param_one_name,param_one_value,gravityspy_id,label,sample_type,url1,url2,url3,url4
0,1.134216e+09,L1,1134216192,931639909,1134216192,832031011,0.18750,Omicron,0,21,1337.695312,1120.043213,573.363953,GDS-CALIB_STRAIN,1.197650e-22,7.51139,0,0,0,phase,-2.72902,zmIdpucyOG,Whistle,train,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...
1,1.129360e+09,L1,1129359781,558593034,1129359781,47851085,0.94238,Omicron,0,107,654.747681,1823.073364,3426.576416,GDS-CALIB_STRAIN,8.255850e-23,9.63013,0,0,0,phase,1.10682,zWFRqqDxwv,Whistle,test,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...
2,1.127425e+09,L1,1127425468,976317882,1127425468,960937023,0.04688,Omicron,0,218,1337.827515,2024.177490,3778.701904,GDS-CALIB_STRAIN,9.762940e-22,15.37104,0,0,0,phase,-0.83099,zKCTakFVcf,Whistle,train,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...
3,1.132637e+09,L1,1132636755,365233898,1132636754,951172113,0.82422,Omicron,0,88,1182.974609,3981.774658,7492.717773,GDS-CALIB_STRAIN,1.462120e-22,10.32116,0,0,0,phase,0.76242,z14BdoiFZS,Whistle,validation,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...
4,1.132036e+09,L1,1132035853,197264909,1132035852,933837890,2.00366,Omicron,0,16,1456.511353,3979.026855,7942.912109,GDS-CALIB_STRAIN,4.038300e-22,14.38016,0,0,0,phase,-0.31161,yyjqLCtAmO,Whistle,validation,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...


In [9]:
df.label.value_counts()


Blip                   1821
Koi_Fish                706
Low_Frequency_Burst     621
Light_Modulation        512
Power_Line              449
Low_Frequency_Lines     447
Extremely_Loud          447
Scattered_Light         443
Violin_Mode             412
Scratchy                337
1080Lines               328
Whistle                 299
Helix                   279
Repeating_Blips         263
No_Glitch               150
Tomte                   103
None_of_the_Above        81
1400Ripples              81
Chirp                    60
Air_Compressor           58
Wandering_Line           42
Paired_Doves             27
Name: label, dtype: int64

In [10]:
classes_list=df.label.value_counts().index
print(classes_list)

Index(['Blip', 'Koi_Fish', 'Low_Frequency_Burst', 'Light_Modulation',
       'Power_Line', 'Low_Frequency_Lines', 'Extremely_Loud',
       'Scattered_Light', 'Violin_Mode', 'Scratchy', '1080Lines', 'Whistle',
       'Helix', 'Repeating_Blips', 'No_Glitch', 'Tomte', 'None_of_the_Above',
       '1400Ripples', 'Chirp', 'Air_Compressor', 'Wandering_Line',
       'Paired_Doves'],
      dtype='object')


In [11]:
training_batch_size = 64
validation_batch_size = 32
img_dim = 250

train_generator = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_dim,img_dim),
  batch_size=training_batch_size)

#cant invert or rotate or do variations due to the distortion which can be caused to the image


Found 22348 files belonging to 22 classes.
Using 17879 files for training.


In [12]:
validation_generator = tf.keras.preprocessing.image_dataset_from_directory(
  validation_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_dim, img_dim),
  batch_size=validation_batch_size)


Found 4800 files belonging to 22 classes.
Using 960 files for validation.


In [13]:
class_names = train_generator.class_names
print(class_names)


['1080Lines', '1400Ripples', 'Air_Compressor', 'Blip', 'Chirp', 'Extremely_Loud', 'Helix', 'Koi_Fish', 'Light_Modulation', 'Low_Frequency_Burst', 'Low_Frequency_Lines', 'No_Glitch', 'None_of_the_Above', 'Paired_Doves', 'Power_Line', 'Repeating_Blips', 'Scattered_Light', 'Scratchy', 'Tomte', 'Violin_Mode', 'Wandering_Line', 'Whistle']


In [14]:
for image_batch, labels_batch in train_generator:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


(64, 250, 250, 3)
(64,)


In [15]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)


In [16]:
normalized_ds = train_generator.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.9921569


In [17]:
num_classes = 22

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_dim, img_dim, 3)),
  layers.Conv2D(128, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(num_classes)
])


In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [19]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 250, 250, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 250, 250, 128)     9728      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 125, 125, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 128)     409728    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 492032)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               1

In [20]:
epochs=10
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  epochs=epochs
)


Epoch 1/10
280/280 [==============================] - 358s 1s/step - loss: 0.7812 - accuracy: 0.8348 - val_loss: 0.2729 - val_accuracy: 0.9094
Epoch 2/10
280/280 [==============================] - 352s 1s/step - loss: 0.1630 - accuracy: 0.9543 - val_loss: 0.2702 - val_accuracy: 0.9240
Epoch 3/10
280/280 [==============================] - 349s 1s/step - loss: 0.0954 - accuracy: 0.9737 - val_loss: 0.3155 - val_accuracy: 0.9323
Epoch 4/10
280/280 [==============================] - 349s 1s/step - loss: 0.0703 - accuracy: 0.9810 - val_loss: 0.3216 - val_accuracy: 0.9312
Epoch 5/10
280/280 [==============================] - 350s 1s/step - loss: 0.0650 - accuracy: 0.9843 - val_loss: 0.3736 - val_accuracy: 0.9281
Epoch 6/10
280/280 [==============================] - 346s 1s/step - loss: 0.0564 - accuracy: 0.9856 - val_loss: 0.4707 - val_accuracy: 0.9333
Epoch 7/10
280/280 [==============================] - 345s 1s/step - loss: 0.0474 - accuracy: 0.9886 - val_loss: 0.4347 - val_accuracy: 0.9177

In [57]:

test_generator = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  subset=None,
  seed=123,
  image_size=(img_dim,img_dim),
  label_mode='int',
  batch_size=training_batch_size)

Found 4720 files belonging to 22 classes.


In [58]:

predictions = model.predict(test_generator, steps = 74, verbose = 1)

74/74 [==============================] - 50s 669ms/step


In [59]:
model.evaluate(test_generator,steps=74)

74/74 [==============================] - 50s 670ms/step - loss: 0.4697 - accuracy: 0.9343


[0.46967050433158875, 0.9343220591545105]